In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [4]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [107]:
# Example 1
x = [2.0, 3.0, -1.0] # inputs
n = MLP(3, [4, 4, 1]) # MLP(inputs, layers=(4 node layer, 4 node layer, 1 node layer - output))
n(x) # passing the inputs to the Multi Layer Perceptron

Value(data=0.1075793441308745)

In [108]:
# Example 2
xs = [
  [2.0, 3.0, -1.0], # expected result = 1.0
  [3.0, -1.0, 0.5], # expected result = -1.0
  [0.5, 1.0, 1.0], # expected result = -1.0
  [1.0, 1.0, -1.0], # expected result = 1.0
] # list of groups of inputs
ys = [1.0, -1.0, -1.0, 1.0] # desired targets for each group of inputs

ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
ypred

[Value(data=0.1075793441308745),
 Value(data=0.28684860123742467),
 Value(data=0.18248596262923636),
 Value(data=0.318379834859191)]

In [109]:
# Calculating Loss value for example 2 - using square function for loss
list_of_losses = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
# zip ex: a = [1,2,3] b = [a,b,c] zip(a,b) = [[1,a],[2,b],[3,c]]
# ygt means Y ground truth which is the expected result

list_of_losses
# returned a list of losses for each group of inputs

loss = sum(list_of_losses)
# total loss = the sum of all losses of expected results

loss

Value(data=4.315273050870372)

In [110]:
# calculating grade of nodes with derivative 

# because ypred is a list of output nodes of the MLP and we apply - and ** to those nodes which are instances of Value that has a -,** method
# it means that the result of list of losses and loss itself are Value objects

# losses = loss of ys0 + loss of ys1 + loss of ys2 + loss of ys3
# loss of ys0 = object MLP + loss function -> loss function = (ys0 - ygt0)**2 -> object MLP is a graph with the output node ys0

# doing backward() on loss will spread the gradient (derivate) to the loss function but also to the MLP
# due to that we have, 1) how much the nodes affect the loss function, but also, 2) how much the nodes affect the output node ys0

loss.backward() # always do one calculation per MLP creation because derivates add up and can acumulate
n.layers[0].neurons[0].w[0].grad
# how much this affects the loss or the output results
# first layer, first neuron, first weight (one weight per input on a neuron, each neuron has 3 inputs), gradient derivative

# the goal is to get a prediction or results as close as the ygt y ground truth value (expected result) to reduce loss as close to zero we can
# if we make a Weight with a possitive gradient higher we will get a higher output
# if we make a Weight with a negative gradient lower we will get a lower output
# if the output is closer to the expected result we will get a lower loss
# if the output is farther from the expected result we will get a higher loss

-0.045665039322060993

In [111]:
# getting all the parameters of the MLP - values we can interact with to change the output (weights and biases)

n.parameters()
# node1w1,node1w2,node1w3,node1bias,node2w1,node2w2,node2w3,node2bias, ... 
# has all the objects inside we can modify them

[Value(data=-0.7512169222274381),
 Value(data=0.6729080180743277),
 Value(data=-0.48918400002044815),
 Value(data=0.9730013024747246),
 Value(data=0.21636980175800624),
 Value(data=0.14496129518537937),
 Value(data=0.22864601946339325),
 Value(data=-0.4379285074685846),
 Value(data=-0.701628557614598),
 Value(data=0.008820360154472073),
 Value(data=-0.5734679922213968),
 Value(data=-0.5712361149626315),
 Value(data=0.3524453178487954),
 Value(data=-0.8671596991472574),
 Value(data=0.659389515987906),
 Value(data=0.8338393118892582),
 Value(data=-0.5018711601697647),
 Value(data=0.4720333827811458),
 Value(data=0.8881706937158571),
 Value(data=0.14976121913673968),
 Value(data=0.3314562602985882),
 Value(data=-0.8856408805332614),
 Value(data=0.04782015704745546),
 Value(data=0.2772402624125272),
 Value(data=0.37119894426162614),
 Value(data=-0.19822048509306356),
 Value(data=-0.20508553396429852),
 Value(data=-0.8831249016114551),
 Value(data=-0.27054806244723095),
 Value(data=-0.04624

In [112]:
# Gradient decent

# we modify each parameter by a tiny STEP size in the direction of the gradient

# following the direction can get us closer to the loss
print({"g": n.layers[0].neurons[0].w[0].grad, "w": n.layers[0].neurons[0].w[0].data})
# 1) if grad is negative means that making W bigger the lost will be lower
# 2) if grad is possitive means that making W smaller the lost will be lower

# if the amount to add is (step * grad) means:
# 1) if grad is negative, W must get bigger -> (step(negative) * grad(negative)) -> step must be negative to cancel the grad
# 2) if grad is possitive, W must get smaller -> (step(negative) * grad(possitive)) -> step must be negative to take out the grad

# the previous is "following the direction of the gradient"

{'g': -0.045665039322060993, 'w': -0.7512169222274381}


In [113]:
# changing the weights
stepSign = -1

# forward pass
for p in n.parameters():
    p.data += stepSign * 0.01 * p.grad
    
n.layers[0].neurons[0].w[0].data
# new weight

-0.7507602718342175

In [114]:
ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
print(ypred)
# Calculating Loss after changing weights
new_loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
print({"old loss": loss, "new loss": new_loss})
loss = new_loss
loss.backward() # backward pass -> recalculate gradient

[Value(data=0.17972347627624333), Value(data=0.1491367966297844), Value(data=0.14438916264037405), Value(data=0.34021384562940726)]
{'old loss': Value(data=4.315273050870372), 'new loss': Value(data=3.7383132778087647)}


In [115]:
# training the Model multiple times

iterations_training = 10

for x in range(iterations_training):
    # forward pass
    for p in n.parameters():
        p.data += -0.01 * p.grad
    # new loss
    ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
    print(ypred)
    # Calculating Loss after changing weights
    new_loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
    print({"old loss": loss, "new loss": new_loss})
    loss = new_loss
    # backward pass
    loss.backward()